In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree.export import export_graphviz
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedShuffleSplit
import warnings; warnings.simplefilter('ignore')
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier
import pyodbc
from datetime import timedelta
import ast
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.calibration import calibration_curve
%matplotlib inline
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from sklearn.calibration import CalibratedClassifierCV
from scipy import stats
import lightgbm as lgb
from matplotlib import pyplot
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scipy import interp
import pickle
from math import sqrt

# Functions

In [ ]:
#model:fitted model
#inputx:data to be predicted
#inputdata: data to be predicted+labels
#threshold: threshold used to get the predicted label. if predicted prob>threshold, predict 1
#bootstrap: True if we are applying the function on bootstrapped dataset
def getscores(model,inputx,inputdata,threshold,bootstrap=False):
    
    x=inputx.copy()
    data=inputdata.copy()
    data['predicted_prob']=model.predict_proba(x)[:,1]
   
    
    if not bootstrap:
        cali_y1, cali_x1 = calibration_curve(data['ards'],data['predicted_prob'],n_bins=10, normalize=True)
        
        aggregation_functions={'predicted_prob':'max','ards':'max'}
        data2 = data.groupby(['EncounterID']).aggregate(aggregation_functions).reset_index(drop=False)

        #get calibration curve
        cali_y2, cali_x2 = calibration_curve(data2['ards'],data2['predicted_prob'],n_bins=10, normalize=False)

     
    #get predicted label
    data.loc[data['predicted_prob']>=threshold,'predicted']=1
    data.loc[data['predicted_prob']<threshold,'predicted']=0

    #ENCOUNTER level predition
    
    aggregation_functions={'predicted':'max','ards':'max'}
    dataenc = data.groupby(['EncounterID']).aggregate(aggregation_functions).reset_index(drop=False)
    #get list of encounterid that the predicted label and true label are both 1
    agree=dataenc[(dataenc['ards']==1 )&(dataenc['predicted']==1)].EncounterID.unique().tolist()
    
    data=data.sort_values(['EncounterID','time'], ascending=[True,True])
    data_hours=data[data['EncounterID'].isin(agree)]
    data_hours=data_hours[data_hours['predicted']==1]
    

    #get time difference
    data_hours.time=pd.to_datetime(data_hours.time)
    
    data_hours.time=pd.to_datetime(data_hours.time)
    data_hours=data_hours.groupby(['EncounterID'])['time'].first().reset_index(drop=False)
    data_hours=pd.merge(data_hours,ards[['EncounterID','ards_time']],how='left',on='EncounterID')
    data_hours.ards_time=pd.to_datetime(data_hours.ards_time)
    data_hours['diff']=(((data_hours['time']+timedelta(hours=bintest)))-data_hours['ards_time'])/ np.timedelta64(1, 'h')
    data_hours.loc[data_hours['diff']<=0,'earlydiff']=abs(data_hours['diff'])
    data_hours.loc[data_hours['diff']>0,'latediff']=abs(data_hours['diff'])
    #get median time difference when the prediction was made earlier than ards_time
    early_avg_diff=np.nanmedian(data_hours['earlydiff'])
    #get median time difference when the prediction was made later than ards_time
    late_avg_diff=np.nanmedian(data_hours['latediff'])
    #get percentage of early prediction
    earlypct=len(data_hours[data_hours['diff']<=0])/len(data_hours)*100
    #get percentage of late prediction
    latepct=len(data_hours[data_hours['diff']>0])/len(data_hours)*100
    
    #avg_diff=(round(earlypct,2),round(early_avg_diff,2),round(latepct,2),round(late_avg_diff,2))
    avg_diff=np.nanmedian(data_hours['diff'])
    
#     print(len(data_hours))
#     print(len(dataenc[dataenc['ards']==1]))
    if not bootstrap:
        #save the data used to plot time curve
        rows_list = []
        for i in range(-48,49):
            dic1 = {}
            dic1['Time to ards_time(hours)']=i
            dic1['Encounter(%)']=len(data_hours[data_hours['diff']<=i])/len(dataenc[dataenc['ards']==1])*100
            rows_list.append(dic1)

        timecurve = pd.DataFrame(rows_list) 
    
    #get sensitivity, specificity and ppv
    CM = confusion_matrix(dataenc['ards'],dataenc['predicted'])
    tn, fp, fn, tp =CM.ravel()
    
    recall=tp/(tp+fn)
    recall_ad=(tp+2)/(tp+fn+4)
    sensitivity=recall
    sen_ci=((sensitivity-1.96*sqrt(sensitivity*(1-sensitivity)/(tp+fn)))*100,(sensitivity+1.96*sqrt(sensitivity*(1-sensitivity)/(tp+fn)))*100)
    
    sp111=tn/(tn+fp)
    sp111_a=(tn+2)/(tn+fp+4)
    specificity=sp111
    spe_ci=((specificity-1.96*sqrt(specificity*(1-specificity)/(tn+fp)))*100,(specificity+1.96*sqrt(specificity*(1-specificity)/(tn+fp)))*100)
    
    sp111=tp/(tp+fp)
    sp111_a=(tp+2)/(tp+fp+4)
    ppv=sp111
    ppv_ci=((ppv-1.96*sqrt(ppv*(1-ppv)/(tp+fp)))*100,(ppv+1.96*sqrt(ppv*(1-ppv)/(tp+fp)))*100)
    
    
    if not bootstrap:
        return round(sensitivity*100,1), round(specificity*100,1),round(ppv*100,1),avg_diff,(cali_y1,cali_y2),(cali_x1,cali_x2),timecurve
    else:
        return round(sensitivity*100,1), round(specificity*100,1),round(ppv*100,1),avg_diff
    
#get bin rocauc and prc score
def getroc(model,y,x):
    
    fpr, tpr, threshold = metrics.roc_curve(y, model.predict_proba(x)[:,1])
    roc_auc = metrics.auc(fpr, tpr)
    
    #prc
    precision, recall, thresholds = precision_recall_curve(y, model.predict_proba(x)[:,1]) 
    pr_auc = metrics.auc(recall, precision)
    
    return round(roc_auc,3),round(pr_auc,3)

#get encounter rocauc and prc score
def getroc_encounter(model,inputx,inputdata,outputname=0,bootstrap=False):
    x=inputx.copy()
    data=inputdata.copy()
    data['predicted']=model.predict_proba(x)[:,1]
    if not bootstrap:
        data[['EncounterID','time','ards','predicted']].to_csv(PATH4+outputname+'_predicted_proba.csv',index=False)
    
    
    aggregation_functions={'predicted':'max','ards':'max'}
    data = data.groupby(['EncounterID']).aggregate(aggregation_functions).reset_index(drop=False)
   
    fpr, tpr, threshold = metrics.roc_curve(data['ards'],data['predicted'])
    roc_auc = metrics.auc(fpr, tpr)

    #save the data to plot rocauc curve
    if outputname!=0 and not bootstrap:
        output=pd.DataFrame(columns=['fpr','tpr'])
        output['fpr']=fpr
        output['tpr']=tpr
        output.to_csv(PATH4+outputname+'_rocauc.csv',index=False)

    precision, recall, thresholds = precision_recall_curve(data['ards'], data['predicted']) 
    #retrieve probability of being 1(in second column of probs_y)
    pr_auc = metrics.auc(recall, precision)
    
    #save the data to plot precision recall curve
    if outputname!=0 and not bootstrap:
        output=pd.DataFrame(columns=['precision','recall'])
        output['precision']=precision
        output['recall']=recall
        output.to_csv(PATH4+outputname+'_prc.csv',index=False)

    if not bootstrap:
        #plot Precision-Recall vs Threshold Chart
        plt.title("Precision-Recall vs Threshold Chart")
        plt.plot(thresholds, precision[:-1] , "b--", label="Precision")
        plt.plot(thresholds, recall[:-1], "r--", label="Recall")
        plt.ylabel("Precision, Recall")
        plt.xlabel("Threshold")
        plt.legend(loc="lower left")
        plt.ylim([0,1])
        plt.show()
    
  
    return round(roc_auc, 3),round(pr_auc,3)



In [ ]:
# Assuming there is more than two columns to the data, you can just read in
# the two columns of data, try to sort it, and then save the index of the
# sorted dataframe. Then you can read in the original data and rearrange it
# using the sorted index.
def sortdf(data,temp,colstosort):

    asc=[True]*len(colstosort)
    temp=temp.sort_values(colstosort, ascending=asc)

    index=temp.index

    data=data.reindex(index)

    return data

In [ ]:
#parameters:
#model: sklearn model with defined hyperparameters
#inputtraindata: the data used to train the model (binned every 6 hours)
#inputtraindata2: the data used to test the model (binned every 2 hours)
#lda: True if the model is lda
#useweight: True if we want to use sampleweight
def crossvalidate(model,inputtraindata,inputtraindata2,lda=False,useweight=True):
    traindata=inputtraindata.copy()
    traindata=traindata.drop('time',1)
    
    traindata2=inputtraindata2.copy()
    traindata2=traindata2.drop('time',1)

    #split the data by patientid
    split=traindata[['PatientID','ards']].groupby(['PatientID']).sum().reset_index()
    split.loc[split['ards']>0,'ards']=1

    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1)
    skf.get_n_splits(split['PatientID'], split['ards'])

    X,y=split['PatientID'], split['ards']
    final=0
    i=1
    #inner cross validation
    for train_index, test_index in skf.split(X, y):
        print('inner',i)
        i+=1
        #print("TRAIN:", train_index, "TEST:", test_index)
        trainset, testset = X[train_index], X[test_index]
        
        X_train=traindata[traindata['PatientID'].isin(trainset)].drop(['EncounterID','PatientID','ards'],1)
        sampleweight=X_train['sampleweight']
        X_train=X_train.drop('sampleweight',1)
        y_train=traindata[traindata['PatientID'].isin(trainset)]['ards']

        X_val=traindata2[traindata2['PatientID'].isin(testset)].drop(['EncounterID','PatientID','ards','sampleweight'],1)
        y_val=traindata2[traindata2['PatientID'].isin(testset)]['ards']

        sc = StandardScaler()  

        X_train_sc= sc.fit_transform(X_train)
        X_val_sc= sc.transform (X_val)

        if lda:
            model.fit(X_train_sc, y_train)

        else:
            if useweight:
                model.fit(X_train_sc, y_train,sample_weight=sampleweight)
            else:
                model.fit(X_train_sc, y_train)
                
        roc,prc=getroc(model,y_val,X_val_sc)
        final=final+roc
       
        
    final=final/5
    print(final)
     

    return final

#model: fitted model
#inputx: data to be predicted
#inputdata: data to be predicted+true labels
def get_threshold(model,inputx,inputdata):
    x=inputx.copy()
    data=inputdata.copy()
    #get predicted probability
    data['predicted_prob']=model.predict_proba(x)[:,1]
    
    #get each encounter's max predicted_probability and labels
    #ards==1 if the encounter had had ards, otherwise 0
    aggregation_functions={'predicted_prob':'max','ards':'max'}
    data = data.groupby(['EncounterID']).aggregate(aggregation_functions).reset_index(drop=False)
    
    #get list of precision, recall, and corresponding thresholds
    precision, recall, thresholds = precision_recall_curve(data['ards'], data['predicted_prob']) 

    
    temp=pd.DataFrame(columns=['precision', 'recall', 'thresholds'])
    temp['precision']=precision[: -1]
    temp['recall']=recall[: -1]
    temp['thresholds']=thresholds
    temp=temp.sort_values(['recall'], ascending=[True])
    temp=temp[temp['recall']>=0.85]
    
    if len(temp)>0:
        final_threshold=temp['thresholds'].iloc[0]
    else:
        final_threshold=0
     
    return final_threshold

In [ ]:
def bootstrap(data):
    dfs = [x for _, x in data.groupby('EncounterID')]
    dfs2 = [random.choice(dfs) for _ in dfs]
    data = pd.concat(dfs2)
    #data=data.iloc[np.random.choice(len(data), size=len(data))]
    return data

def getci(data):
    i=0
    final={'rocauc':[],'prauc':[],'rocauc_enc':[],'prc_enc':[],'sen':[],'spe':[],'ppv':[],'avgdiff':[]}
    while i<=1000:
        print(i)
        i+=1
        temp=bootstrap(data)
        X_sc= sc.transform(temp.drop(['EncounterID','PatientID','ards','time','sampleweight'],1))
        rocauc,prauc=getroc(clf,temp['ards'],X_sc)
        rocauc_encounter,prauc_encounter=getroc_encounter(clf,X_sc,temp,outputname=0,bootstrap=True)
        sensitivity,specificity, ppv,timediff=getscores(clf,X_sc,temp,final_threshold,bootstrap=True)

        final['rocauc'].append(rocauc)
        final['prauc'].append(prauc)
        final['rocauc_enc'].append(rocauc_encounter)
        final['prc_enc'].append(prauc_encounter)
        final['sen'].append(sensitivity)
        final['spe'].append(specificity)
        final['ppv'].append(ppv)
        final['avgdiff'].append(timediff)
    
    for k in final.keys():
        final[k].sort()
        final[k]=np.percentile(final[k], [2.5, 97.5])  
        
    return final

# Set paramters

In [ ]:
test_on_intubated=True
use_sample_weight=True
choose_model='logistic regression'
classweight=None #None or 'balanced'
ifcalibrate=False
#what data we want to use: structured, structured+unstructured, structured+unstructured+order, or structured+unstructured+clinicalnotes
file='structured_allmedian'

In [ ]:
PATH1='Z:\patient-adjudication-results\\'
PATH2='Z:\project-datasets\ARDS\ml_algorithms\\final_datasets_testing\\'
PATH3='Z:\project-datasets\ARDS\ml_algorithms\\'
PATH4='Z:\project-datasets\ARDS\ml_algorithms\model_outputs_testing\\'

# Read in data

In [ ]:
ards=pd.read_csv(PATH1+'current-ards-review-results_2_25_2020.csv',dtype={'mrn': str})
ards.rename(columns={'encounterid':'EncounterID'},inplace=True)
ards.loc[ards['ards_time']=='.','ards_time']=np.nan

In [ ]:
#We've settled on using data binned every 6h for training, and data binned every 2h for validation and testing
#This script can run on any training data in the format of 'EncounterID','PatientID','ards','time','sampleweight',features
if 'only' in file:
    bintrain=0
    bintest=0
    filename1=PATH2+file+'_train.csv'
    filename2=PATH2+file+'_train.csv'
else:
    filename1=PATH2+file+'_6Htrain.csv'
    filename2=PATH2+file+'_2Htrain.csv'
    bintrain=int(filename1.split('_')[-1][0])
    bintest=int(filename2.split('_')[-1][0])
    
train_str=pd.read_csv(filename1)
train_str2=pd.read_csv(filename2)

In [ ]:
#get list of encounters that have been intubated 
structured=pd.read_csv(PATH2+'structured_data.csv')
intubated=structured[structured['support']=='invasive']
intubated_encounters=intubated.EncounterID.unique().tolist()

# Cross validation

In [ ]:
#logistic regression

In [ ]:
#logistic regression
parameters1 = {'penalty' : ['l1'] ,'C': [ 0.005,0.01,0.05,0.1],'class_weight':[classweight],'n_jobs':[-1],'random_state':[1234]}

#random forest
parameters2 = {'bootstrap': [True],
'max_depth': [8,16,None],
'max_features': ['auto'],
'min_samples_leaf': [10],
'min_samples_split': [50,100],
'n_estimators': [400],
'class_weight':[classweight],
'random_state':[1234]}

#lightgbm
parameters3={'num_leaves':[8,16,32],
             'max_depth':[4,8,-1],
             'min_data_in_leaf':[50,100,200],
             'n_estimators':[400],
             'learning_rate':[0.01],
             'objective':['binary'],
             'subsample':[0.3,0.5],
             'colsample_bytree':[0.3,0.5],
             'random_state':[1234],
             'n_jobs':[-1],
             'reg_alpha':[0.3,0.5],
             'silent':[False],
             'class_weight':[classweight]
            }

#lda
parameters4={'solver':['lsqr'],
            'shrinkage':[None,'auto',0.2,0.4,0.6,0.8],
            'n_components':[None]}

paramdic={'logistic regression':parameters1,'random forest':parameters2,'lightgbm':parameters3,'lda':parameters4}

In [ ]:
#train model using 5 fold inner cross validation on the entire training set
        
#get list of hyperparameter combinations
grid=list(ParameterGrid(paramdic[choose_model]))
model=choose_model
        
#inner corss validation
#crossvalidate() returns the averaged bin rocauc score after running 5 folds inner cross validation
best_parameters={}
for p in grid:
    print(p)
    if model=='logistic regression':
        best_parameters[str(p)]=crossvalidate(LogisticRegression(**p),train_str,train_str2,useweight=use_sample_weight)
    elif model=='lda':
        best_parameters[str(p)]=crossvalidate(LinearDiscriminantAnalysis(**p),train_str,train_str2,lda=True,useweight=use_sample_weight)
    elif model=='random forest':
        best_parameters[str(p)]=crossvalidate(RandomForestClassifier(**p),train_str,train_str2,useweight=use_sample_weight)
    elif model=='lightgbm':
        best_parameters[str(p)]=crossvalidate(lgb.LGBMClassifier(**p),train_str,train_str2,useweight=use_sample_weight)
            
                

#find the set of hyperparameters with the highest bin rocauc, retrain model on the full training set 
       
best=max(best_parameters, key=best_parameters.get)

sampleweight=train_str['sampleweight']

#standardize
sc = StandardScaler()  
X_train_sc= sc.fit_transform(train_str.drop(['EncounterID','PatientID','ards','time','sampleweight'],1))
X_train_sc2= sc.transform(train_str2.drop(['EncounterID','PatientID','ards','time','sampleweight'],1))


#define models
if model=='logistic regression':
    clf = LogisticRegression(**ast.literal_eval(best))  
           
elif model=='lda':
    clf = LinearDiscriminantAnalysis(**ast.literal_eval(best))

elif model=='random forest':
    clf = RandomForestClassifier(**ast.literal_eval(best))

elif model=='lightgbm':
    clf = lgb.LGBMClassifier(**ast.literal_eval(best))
        
#True if we want to calibrate the model
if ifcalibrate:
    clf=CalibratedClassifierCV(clf, method='sigmoid', cv=5)
        
#Fit the model to the entire training set
#True if we want to use sampleweight
if use_sample_weight and model!='lda':
    clf.fit(X_train_sc, train_str['ards'],sampleweight)
else:
    clf.fit(X_train_sc, train_str['ards'])

#output coefficients for logistic regression model
if model=='logistic regression':
    names=train_str.drop(['EncounterID','PatientID','ards','time','sampleweight'],1).columns
    coefficients= pd.concat([pd.DataFrame(names),pd.DataFrame(np.transpose(clf.coef_))], axis = 1)
    coefficients.to_csv(PATH4+'coefficients_'+choose_model+'_'+file+'.csv',index=False)
    
# Save to file in the current working directory
pkl_filename = PATH4+choose_model+'_'+file+"_model.pkl"
with open(pkl_filename, 'wb') as file1:
    pickle.dump(clf, file1)

In [ ]:
print('best hyperparameters:', best)

In [ ]:
#training score

In [ ]:
if test_on_intubated:
    sc = StandardScaler()  
    train_str2=train_str2[train_str2['EncounterID'].isin(intubated_encounters)]
    _= sc.fit_transform(train_str.drop(['EncounterID','PatientID','ards','time','sampleweight'],1))
    X_train_sc2= sc.transform(train_str2.drop(['EncounterID','PatientID','ards','time','sampleweight'],1))

In [ ]:
#get the threshold so the sensitivity==85%
train_threshold=get_threshold(clf,X_train_sc2,train_str2)
print('threshold:',train_threshold)

In [ ]:
train_str.shape

In [ ]:
rocauc,prauc=getroc(clf,train_str2['ards'],X_train_sc2)
rocauc_encounter,prauc_encounter=getroc_encounter(clf,X_train_sc2,train_str2,outputname=file+' '+choose_model+' train')
sensitivity,specificity, ppv,timediff,caliy,calix,timecurve=getscores(clf,X_train_sc2,train_str2,train_threshold)

In [ ]:
#calibration plot encounter level
for i in [0,1]:
    fig = plt.figure(1, figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)

    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")


    ax1.plot(calix[i], caliy[i], "s-",
                     label="%s" % (choose_model))

    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title('Calibration plots  (reliability curve)')
    plt.show()
    if i==0:
        fig.savefig(PATH4+file+' '+choose_model+"_train_calibration_bin.pdf", bbox_inches='tight')
    else:
        fig.savefig(PATH4+file+' '+choose_model+"_train_calibration_encounter.pdf", bbox_inches='tight')

In [ ]:
'train score: ', rocauc,prauc

In [ ]:
'train score: ',rocauc_encounter,prauc_encounter

In [ ]:
'train score: ',sensitivity, specificity, ppv

In [ ]:
'train score: ', timediff

# Testing

In [ ]:
test_on_intubated=True
choose_model='logistic regression'
file='structured'
PATH1='Z:\patient-adjudication-results\\'
PATH2='Z:\project-datasets\ARDS\ml_algorithms\\final_datasets_alternative\\'
PATH3='Z:\project-datasets\ARDS\ml_algorithms\\'
PATH4='Z:\project-datasets\ARDS\ml_algorithms\model_outputs_alternative\\'

In [ ]:
#read in test data

if 'only' in file:
    bintrain=0
    bintest=0
    filename1=PATH2+file+'_test.csv'
    filename2=PATH2+file+'_train.csv'
else:
    
    filename1=PATH2+file+'_2Htest.csv'
    filename2=PATH2+file+'_6Htrain.csv'
    bintest=int(filename1.split('_')[-1][0])
    bintrain=int(filename2.split('_')[-1][0])

testdata=pd.read_csv(filename1)
traindata=pd.read_csv(filename2)
ards=pd.read_csv(PATH1+'current-ards-review-results_2_25_2020.csv',dtype={'mrn': str})
ards.rename(columns={'encounterid':'EncounterID'},inplace=True)
ards.loc[ards['ards_time']=='.','ards_time']=np.nan

In [ ]:
#get list of encounters that have been intubated 
structured=pd.read_csv(PATH2+'structured_data.csv')
intubated=structured[structured['support']=='invasive']
intubated_encounters=intubated.EncounterID.unique().tolist()
if test_on_intubated:
    testdata=testdata[testdata['EncounterID'].isin(intubated_encounters)]

In [ ]:
#standardize
sc = StandardScaler()  
_=sc.fit_transform(traindata.drop(['EncounterID','PatientID','ards','time','sampleweight'],1))
X_test_sc= sc.transform(testdata.drop(['EncounterID','PatientID','ards','time','sampleweight'],1))

In [ ]:
# load the model from disk
pkl_filename = PATH4+choose_model+'_'+file+"_model.pkl"
clf= pickle.load(open(pkl_filename, 'rb'))

In [ ]:
#get threshold
final_threshold=get_threshold(clf,X_test_sc,testdata)

In [ ]:
print(final_threshold)

In [ ]:
#get the test scores
rocauc,prauc=getroc(clf,testdata['ards'],X_test_sc)
rocauc_encounter,prauc_encounter=getroc_encounter(clf,X_test_sc,testdata,outputname=file+' '+choose_model)
sensitivity,specificity,ppv,timediff,caliy,calix,timecurve=getscores(clf,X_test_sc,testdata,final_threshold)
timecurve.to_csv(PATH4+file+' '+choose_model+'_timecurve.csv',index=False)

In [ ]:
testdata.shape

In [ ]:
'test score: ',rocauc,prauc

In [ ]:
'test score: ',rocauc_encounter,prauc_encounter

In [ ]:
'test score: ',sensitivity, specificity,ppv

In [ ]:
'test score: ',timediff

In [ ]:
#calibration plot at encounter level
for i in [0,1]:
    fig = plt.figure(1, figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)

    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")


    ax1.plot(calix[i], caliy[i], "s-",
                     label="%s" % (choose_model))

    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title('Calibration plots  (reliability curve)')
    plt.show()
    if i==0:
        fig.savefig(PATH4+file+' '+choose_model+"_calibration_bin.pdf", bbox_inches='tight')
    else:
        fig.savefig(PATH4+file+' '+choose_model+"_calibration_encounter.pdf", bbox_inches='tight')


In [ ]:
###time curve
filenames=[PATH4+file+' '+choose_model+'_timecurve.csv']
n_classes=len(filenames)

fig = plt.figure()
ax = fig.gca()
ax.set_xticks(np.arange(-48, 48, 12))
ax.set_yticks(np.arange(0, 100, 10))
plt.xlabel('Time to ards_time(hours)')
plt.ylabel('Encounter(%)')
colors=['red','green','blue','orange','purple','gray']
for i in range(n_classes):
    temp=pd.read_csv(filenames[i])
    plt.plot(temp['Time to ards_time(hours)'], temp['Encounter(%)'],
             label=filenames[i].split('\\')[-1].split('_')[0],color=colors[i],linewidth=0.8,linestyle='-')
    plt.legend(loc="lower right")

plt.grid()
plt.show()
fig.savefig(PATH4+file+' '+choose_model+"_timecurve.pdf", bbox_inches='tight')

In [ ]:
#roc auc
filenames=[PATH4+file+' '+choose_model+'_rocauc.csv']
n_classes=len(filenames)
fpr={}
tpr={}
roc_auc ={}

for i in range(n_classes):
    temp=pd.read_csv(filenames[i])
    fpr[i], tpr[i]=temp['fpr'],temp['tpr']
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
    

# Plot all ROC curves
fig=plt.figure()

colors=['red','green','blue','orange','purple','gray']
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i],
             label=filenames[i].split('\\')[-1].split('_')[0]+'(area = {0:0.4f})'
                   ''.format(roc_auc[i]),
             color=colors[i],  linewidth=0.8,linestyle='-')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC AUC Curve')
plt.legend(loc="best")
plt.show()
fig.savefig(PATH4+file+' '+choose_model+"_rocauc.pdf", bbox_inches='tight')

In [ ]:
#prc
filenames=[PATH4+file+' '+choose_model+'_prc.csv']
n_classes=len(filenames)
precision={}
recall={}
prc ={}

for i in range(n_classes):
    temp=pd.read_csv(filenames[i])
    precision[i], recall[i]=temp['precision'],temp['recall']
    prc[i] = metrics.auc( recall[i],precision[i])
    

# Plot all ROC curves
fig=plt.figure()

colors=['red','green','blue','orange','purple','gray']
for i in range(n_classes):
    plt.plot(recall[i],precision[i], 
             label=filenames[i].split('\\')[-1].split('_')[0]+'(area = {0:0.4f})'
                   ''.format(prc[i]),
             color=colors[i],  linewidth=0.8,linestyle='-')


plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall Curve')
plt.legend(loc="best")
plt.show()
fig.savefig(PATH4+file+' '+choose_model+"_prc.pdf", bbox_inches='tight')

In [ ]:
#get 95% confidence interval
CI=getci(testdata)

#print out 95% confidence interval
print('95% confidence interval for test scores')
for k in CI.keys():
    print(k,' ',CI[k])